In [1]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  data = "/content/drive/MyDrive/ComputationalGenomics/"
except:
  data = "/Users/fionshiau/Library/CloudStorage/GoogleDrive-fioncshiau@gmail.com/My Drive/ComputationalGenomics"


In [2]:
%%capture
!pip install pomegranate;
!pip install watermark;

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style('whitegrid')
import numpy as np
import pandas as pd
import os
import glob
from hmmlearn.hmm import MultinomialHMM

from pomegranate import *

np.random.seed(0)
np.set_printoptions(suppress=True)

%load_ext watermark
%watermark -m -n -p numpy,scipy,pomegranate

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
local_path = "/Users/fionshiau/comp_geno"
obs = pd.read_csv(os.path.join(local_path,'merged','merged_1.csv'),index_col=0)

In [ ]:
chr1 = obs.loc[obs.chr=='chr1',:]
chr1 = chr1.iloc[:,3:]
chr1 = chr1.to_numpy()

In [ ]:
chr1.shape

(1246254, 9)

In [ ]:
chr1.sum(axis=0)

array([ 4869, 14011,  2381,  4831,  6186,  2737, 95550, 27540,  1032])

In [ ]:
help(HiddenMarkovModel.from_samples)

Help on built-in function from_samples:

from_samples(...) method of builtins.type instance
    Learn the transitions and emissions of a model directly from data.
    
    This method will learn both the transition matrix, emission distributions,
    and start probabilities for each state. This will only return a dense
    graph without any silent states or explicit transitions to an end state.
    Currently all components must be defined as the same distribution, but
    soon this restriction will be removed.
    
    If learning a multinomial HMM over discrete characters, the initial
    emisison probabilities are initialized randomly. If learning a
    continuous valued HMM, such as a Gaussian HMM, then kmeans clustering
    is used first to identify initial clusters.
    
    Regardless of the type of model, the transition matrix and start
    probabilities are initialized uniformly. Then the specified learning
    algorithm (Baum-Welch recommended) is used to refine the parameters

In [ ]:
model = HiddenMarkovModel.from_samples(JointProbabilityTable, n_components = 100, X=chr1, n_jobs = -1)

/var/folders/zb/k8zh8whx36g93snxwz8xsqq00000gn/T/ipykernel_62451/2431243336.py:1: RuntimeWarning: invalid value encountered in divide
  model = HiddenMarkovModel.from_samples(IndependentComponentsDistribution,


ValueError: probabilities contain NaN

In [302]:
model = MultinomialHMM(n_components = 500,random_state = 0,n_trials=9,n_iter=400,tol=-1e4)

MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


In [250]:
chr1_mut = np.vstack([np.vstack([chr1[:,c],(chr1[:,c]==0).astype(int)]) for c in range(chr1.shape[1])],dtype=int)

In [251]:
chr1_mut.shape

(18, 1246254)

In [252]:
chr1_mut.sum(axis=1)

array([  44559, 1201695,   48961, 1197293,   18491, 1227763,   37756,
       1208498,   61010, 1185244,   26126, 1220128, 1111450,  134804,
        111606, 1134648,    8557, 1237697])

In [301]:
obs_mut = obs.iloc[:,3:].to_numpy()
obs_mut = np.vstack([np.vstack([obs_mut[:,c],(obs_mut[:,c]==0).astype(int)]) for c in range(obs_mut.shape[1])],dtype=int)

In [303]:
model.fit(obs_mut.T, len(obs_mut))

/var/folders/zb/k8zh8whx36g93snxwz8xsqq00000gn/T/ipykernel_62451/943843744.py:1: DeprecationWarning: less that 14405178 samples in lengths array 18; support for silently dropping samples is deprecated and will be removed
  model.fit(obs_mut.T, len(obs_mut))


MultinomialHMM(n_components=500, n_iter=400, n_trials=9,
               random_state=RandomState(MT19937) at 0x205535840, tol=-10000.0)

In [276]:
logprob, received = model.decode(chr1_mut.T)

In [277]:
logprob

-inf

In [297]:
help(model)

Help on MultinomialHMM in module hmmlearn.hmm object:

class MultinomialHMM(hmmlearn.base.BaseHMM)
 |  MultinomialHMM(n_components=1, n_trials=None, startprob_prior=1.0, transmat_prior=1.0, algorithm='viterbi', random_state=None, n_iter=10, tol=0.01, verbose=False, params='ste', init_params='ste', implementation='log')
 |  
 |  Hidden Markov Model with multinomial emissions
 |  
 |  Attributes
 |  ----------
 |  n_features : int
 |      Number of possible symbols emitted by the model (in the samples).
 |  
 |  monitor_ : ConvergenceMonitor
 |      Monitor object used to check the convergence of EM.
 |  
 |  startprob_ : array, shape (n_components, )
 |      Initial state occupation distribution.
 |  
 |  transmat_ : array, shape (n_components, n_components)
 |      Matrix of transition probabilities between states.
 |  
 |  emissionprob_ : array, shape (n_components, n_features)
 |      Probability of emitting a given symbol when in each state.
 |  
 |  Examples
 |  --------
 |  >>> fr

In [304]:
model.emissionprob_.sum(axis=0)

array([ 0.        , 55.55555556,  0.        , 55.55555556,  0.        ,
       55.55555556,  0.        , 55.55555556,  0.        , 55.55555556,
        0.        , 55.55555556,  0.        , 55.55555556,  0.        ,
       55.55555556,  0.        , 55.55555556])

In [306]:
model.monitor_

ConvergenceMonitor(
    history=[-257.26974385281085, -125.51748688700107, -125.51748688700104, -125.51748688700106, -125.51748688700106, -125.51748688700106, -125.51748688700104, -125.51748688700106, -125.51748688700106, -125.51748688700106, -125.51748688700106, -125.51748688700106, -125.51748688700104, -125.51748688700106, -125.51748688700106, -125.51748688700106, -125.51748688700106, -125.51748688700106, -125.51748688700106, -125.51748688700104, -125.51748688700106, -125.51748688700106, -125.51748688700107, -125.51748688700107, -125.51748688700107, -125.51748688700104, -125.51748688700106, -125.51748688700106, -125.51748688700107, -125.51748688700106, -125.51748688700106, -125.51748688700106, -125.51748688700104, -125.51748688700104, -125.51748688700107, -125.51748688700106, -125.51748688700106, -125.51748688700107, -125.51748688700106, -125.51748688700106, -125.51748688700107, -125.51748688700106, -125.51748688700107, -125.51748688700106, -125.51748688700107, -125.51748688700107, -

In [130]:
obs.columns

Index(['chr', 'start', 'end', 'H3K4me2', 'H3K9Ac', 'H3K4me3', 'ATAC-seq',
       'H3K4me1', 'H3K27Ac', 'HiC', 'H3K27me3', 'RNA-Pol_ChIPseq'],
      dtype='object')

In [ ]:
dist = MultivariateGaussianDistribution.from_samples(obs.iloc[:, 3:].to_numpy())


In [ ]:
dist

{
    "class" : "Distribution",
    "name" : "MultivariateGaussianDistribution",
    "parameters" : [
        [
            0.12134942032649648,
            0.12134935090701413,
            0.12137024617120316,
            0.12140370636169855,
            0.12137788231426228,
            0.12140453939548682,
            0.12139523718485117,
            0.12139607021863943,
            0.12137024617120316
        ],
        [
            [
                0.10662373851291976,
                0.10128705481136609,
                0.09666722115325244,
                0.09392595058925125,
                0.09231174922251781,
                0.09116767462813201,
                0.090252327439942,
                0.08947084065838028,
                0.08879137262156474
            ],
            [
                0.10128705481136609,
                0.10662368594146047,
                0.10132902549681877,
                0.09670398786193933,
                0.0939068785140741,
              

In [ ]:
#https://github.com/keoughkath/LAD_atlas/blob/main/HMM_QC.ipynb as a reference to use bedgraph files in pomegranate 

states_dict = {
        2:['nonLAD','LAD'],
        3:['nonLAD','T2-LAD','T1-LAD'],
        4:['nonLAD','T3-LAD','T2-LAD','T1-LAD'],
        5:['nonLAD','T4-LAD','T3-LAD','T2-LAD','T1-LAD']
    }

['nonLAD', 'LAD']